In [6]:
import torch
import numpy as np
import torchvision
from torch import nn
from skimage.transform import resize
from skimage.color import rgb2gray
import pandas as pd 
from torch.nn import Linear, ReLU, BCELoss, Conv2d, Module, Sigmoid
from torch.optim import Adam
from sklearn.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm
import time

In [7]:
torch.backends.cudnn.enabled

True

In [8]:
torch.cuda.is_available()

False

In [34]:
def process(df):
    
    #get rid of black images
    df.drop(df[df['Image'].map(np.max) == 0].index,inplace=True)
    
    #get rid of rgb images
    def func(img):
        if len(img.shape)>2:
            return True
        return False
    
    df.drop(df[df['Image'].map(func)].index,inplace=True)
    
    
    #process all the available images; to grayscale,60*60,normalize,proper dimension for pytorch 
    def processImage(px_data):
        
        px_data_scaled = px_data / px_data.max()
        px_data_scaled = resize(px_data, (60, 60), anti_aliasing=True)
        px_data_scaled = px_data_scaled[None,:,:]
        return px_data_scaled    
    
    
    df.loc[:,'Image']=df.apply(lambda x: processImage(x['Image']), axis=1)
    
    #make the labels into bool Alive is True
    df.loc[:,'label'] = df['label'].map(lambda x: x == 'Alive')
    
    return df
    
    

In [35]:
class Dataset(torch.utils.data.Dataset):
    """Pet images dataset."""
    
    @staticmethod
    def from_dataframe(df):
        """
        args:
            petimages : PetImages -- The PetImages object containing the images.
        kwargs:
            size : int -- the size of the canonicalized images.
        """
        data = torch.as_tensor(np.array(df['Image'].tolist(),dtype=np.float32))
        labels = np.array(df['label'],dtype=np.int8)[:,None]

        
        return Dataset(data, labels)

    
    def __init__(self, data, labels):
        # Don't change the constructor
        self.data = data
        self.labels = labels

    def __len__(self):
        """ Return the number of images in the dataset.
        """
        return self.data.shape[0]


    def __getitem__(self, idx):
        """ Return the element corresponding to idx.
        
        args:
            idx : int -- the index of the sample to return
            
        returns: Dict -- A dictionary with two elements; "label" and "image". "label" has the associated label
            and "image" is a (size, size, 3)
        """
        # Convert it to a regular python int.
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return {'label':self.labels[idx],'image':self.data[idx]}
    
def Dataset_load(df):
    return Dataset.from_dataframe(df)

In [36]:
def split(dataset):
    """ Split pet into train and test sets.
    
    args:
        pet : PetDataset -- the PetDataset instance to split.

    kwargs:
        train_count: The number of elements in the training set. The remainder should be in the test set.
    
    return: List[Dataset] -- the list of [train, test] datasets.
    """
    total = len(dataset)
    test = int(np.ceil(0.2*total))
    train = total-test

    
    return torch.utils.data.random_split(dataset,[train,test])

In [37]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 12, 5)
        self.relu1 = nn.ReLU()
        self.avg1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(12, 30, 5)
        self.tanh2 = nn.Tanh()
        self.avg2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1   = nn.Linear(4320, 1)
        self.sigm1 = nn.Sigmoid()
    
    def forward(self, X, debug=False):
        if debug: print(f"Input Shape: {X.shape}")

        X = self.avg1(self.relu1(self.conv1(X)))
        if debug: print(f"Conv1 Shape: {X.shape}")
            
        X = self.avg2(self.tanh2(self.conv2(X)))
        if debug: print(f"Conv1 Shape: {X.shape}")

        X = X.view(X.size(0), -1) # Flatten the shape
        if debug: print(f"Flattened Shape: {X.shape}")

        X = self.sigm1(self.fc1(X))
        if debug: print(f"Output Shape: {X.shape}")

        return X

def count_parameters(model):
    # Count all trainable parameters,
    # from https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/9
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [38]:
mM = Model()
X = torch.zeros(5,1,60,60)
y = mM(X,debug=True)

Input Shape: torch.Size([5, 1, 60, 60])
Conv1 Shape: torch.Size([5, 12, 28, 28])
Conv1 Shape: torch.Size([5, 30, 12, 12])
Flattened Shape: torch.Size([5, 4320])
Output Shape: torch.Size([5, 1])


In [39]:
def training_loop(model, train_dataset, epochs=25, batch_size=500):
    """ Train the model on data
    """
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

#     pass # TODO: Set up
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    

    start_time = time.perf_counter()
    for epoch in range(epochs):
        # If you add the training loss to this variable, it will be printed for you
        epoch_loss = 0.0
        
        for data in train_dataloader:
            output = model(data['image'])
            loss = criterion(output,data['label'].float())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            epoch_loss+= loss.item()
#         pass # TODO:Process all data for each epoch

        epoch += 1
        if epoch % 50 == 0:
            curr_time = time.perf_counter() - start_time
            print(f'[{curr_time:6.1f}/{curr_time/epoch*epochs:6.1f}] Epoch {epoch: <3d} loss: {epoch_loss / len(train_dataloader)} acc: {test_model(model, train_dataset)}')
    print("Done.")

In [40]:
def test_model(model, test_data, batch_size=500):
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
    num_correct = 0
    num_alive = 0
    num_alive_correct = 0
    num_dead = 0
    num_dead_correct = 0
    for data in test_dataloader:
        y_pred = model(data["image"]).round()
        y_actual = data["label"].float()
        num_correct += (y_pred == y_actual).sum()
        for ya,yp in zip(y_actual,y_pred):
            if ya == 1.0:
                num_alive+=1
                if yp == 1.0:
                    num_alive_correct+=1
            else:
                num_dead+=1
                if yp ==0.0:
                    num_dead_correct+=1
    print(num_dead,num_alive)
    return num_correct.item() / len(test_data),(num_alive_correct/num_alive),(num_dead_correct/num_dead)

In [43]:
def trainModel(filename,epoch,batchsize):
    train_df = pd.read_pickle(filename)
    df_train = process(train_df)
    print(len(df_train))
    ds = Dataset_load(df_train)
    train_data, valid_data = tuple(split(ds))

    model = Model()

    training_loop(model, train_data,epochs = epoch,batch_size = batchsize)

    train_acc,alive_acc,dead_acc = test_model(model, train_data)
    print(f"Train accuracy: {train_acc} Alive accuracy: {alive_acc} Dead accuracy: {dead_acc}")

    test_acc,talive_acc,tdead_acc = test_model(model, valid_data)
    print(f"Test accuracy: {test_acc} Alive accuracy: {talive_acc} Dead accuracy: {tdead_acc}")
    
    return model,train_data,valid_data
    
myModel,_,_ = trainModel("train_csv.pkl",100,100)

4345
534 2942
[ 507.8/2031.1] Epoch 50  loss: 0.36179090567997524 acc: (0.8644994246260069, 0.9938817131203264, 0.15168539325842698)
534 2942
[1139.3/2278.6] Epoch 100 loss: 0.3191119134426117 acc: (0.8780207134637514, 0.9942216179469748, 0.23782771535580524)
534 2942
[1594.3/2125.8] Epoch 150 loss: 0.2859384949718203 acc: (0.8866513233601842, 0.9935418082936778, 0.29775280898876405)
534 2942
[2037.6/2037.6] Epoch 200 loss: 0.256148675935609 acc: (0.8944188722669736, 0.9928619986403807, 0.352059925093633)
Done.
534 2942
Train accuracy: 0.8944188722669736 Alive accuracy: 0.9928619986403807 Dead accuracy: 0.352059925093633
136 733
Test accuracy: 0.8734177215189873 Alive accuracy: 0.975443383356071 Dead accuracy: 0.3235294117647059


In [45]:
test_df = pd.read_pickle('test_data.pkl')
df_test = process(test_df)
print(len(df_test))
ds = Dataset_load(df_test)
test_model(myModel,ds)

1088
177 911


(0.8529411764705882, 0.9681668496158068, 0.2598870056497175)